In [1]:
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")

test = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")

In [3]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [4]:
train = train[train['Date']<='2020-04-14']
train['Location'] = train['Province_State'].astype(str) + train['Country_Region'].astype(str)
test['Location'] = test['Province_State'].astype(str) + test['Country_Region'].astype(str)
test = test.merge(train[['ConfirmedCases','Fatalities','Location','Date']],how='left',on=['Location','Date'])
train = train[train['Date']<='2020-04-01']
data = pd.concat([train,test],axis=0)
data = data.sort_values(['Country_Region','Date'])

In [5]:
data.drop(['Id','Province_State','Country_Region'],axis=1,inplace=True)

In [6]:
import gc
del train,test
gc.collect()

40

In [7]:
data

,Date,ConfirmedCases,Fatalities,Location,ForecastId
0,2020-01-22,0.0,0.0,nanAfghanistan,NaN
1,2020-01-23,0.0,0.0,nanAfghanistan,NaN
2,2020-01-24,0.0,0.0,nanAfghanistan,NaN
3,2020-01-25,0.0,0.0,nanAfghanistan,NaN
4,2020-01-26,0.0,0.0,nanAfghanistan,NaN
...,...,...,...,...,...
13454,2020-05-10,NaN,NaN,nanZimbabwe,13455.0
13455,2020-05-11,NaN,NaN,nanZimbabwe,13456.0
13456,2020-05-12,NaN,NaN,nanZimbabwe,13457.0
13457,2020-05-13,NaN,NaN,nanZimbabwe,13458.0


In [8]:
locations = data['Location'].unique()

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=4)
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [10]:
Res = None

for i in tqdm(locations):
    df = data[data['Location']==i].reset_index().drop(['index'],axis=1).reset_index() 
    X_c = poly.fit_transform(np.array(df['index']).reshape(-1,1))
    df[[0,1,2,3,4]] = pd.DataFrame(X_c)
    
    train = df[df['Date']<='2020-04-14'][['ForecastId','ConfirmedCases','Fatalities',0,1,2,3,4]]
    test = df[df['Date']>='2020-04-15'][['ForecastId','ConfirmedCases','Fatalities',0,1,2,3,4]]
    
    
    model = LinearRegression()
    model.fit(train.drop(['ConfirmedCases','ForecastId','Fatalities'],axis=1),train['ConfirmedCases'])
    test['ConfirmedCases']=model.predict(test.drop(['ConfirmedCases','ForecastId','Fatalities'],axis=1))
    
    model = LinearRegression()
    model.fit(train.drop(['ConfirmedCases','ForecastId','Fatalities'],axis=1),train['Fatalities'])
    test['Fatalities']=model.predict(test.drop(['ConfirmedCases','ForecastId','Fatalities'],axis=1))
    
    pred = pd.concat([train[-13:][['ForecastId','ConfirmedCases','Fatalities']],test[['ForecastId','ConfirmedCases','Fatalities']]])
    
    Res = pd.concat([Res,pred],axis=0)

100%|██████████| 313/313 [00:11<00:00, 28.29it/s]


In [11]:
Res['ForecastId'] = Res['ForecastId'].astype(int)

In [12]:
Res.to_csv("submission.csv",index=False)

In [13]:
Res

,ForecastId,ConfirmedCases,Fatalities
71,1,273.000000,6.000000
72,2,281.000000,6.000000
73,3,299.000000,7.000000
74,4,349.000000,7.000000
75,5,367.000000,11.000000
...,...,...,...
109,13455,56.726611,15.404033
110,13456,58.910219,16.151665
111,13457,61.148926,16.925716
112,13458,63.443452,17.726793
